In [82]:
import json
from web3 import Web3
from taxes_abi import *

In [83]:
ganache_url = 'HTTP://127.0.0.1:7545'

In [84]:
web3 = Web3(Web3.HTTPProvider(ganache_url))

In [85]:
web3.eth.defaultAccount = web3.eth.accounts[0] 
print(web3.isConnected())

True


In [86]:
contract_abi = json.loads(abi)
contract_bytecode = bytecode[0]

In [87]:
def deploy_contract(abi, bytecode):
    """
    Deploy the contract using Python and web3.py without relying on Remix
    
    Parameters
    --------------
    abi : abi of the contract
    bytecode : bytecode of the contract
    
    Returns
    --------------
    Address where the contract has been deployed
    """
    
    contract = web3.eth.contract(abi=abi, bytecode=bytecode)
    tx_hash = contract.constructor().transact()
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt.contractAddress


In [88]:
contract_address = deploy_contract(abi=contract_abi, bytecode=contract_bytecode)

print(contract_address)

0x6bdbE1eC3c9EE9Cb31F9138F0e36eC7377D8538B


In [89]:
# Define contract
rubbish_contract = web3.eth.contract(address=contract_address, abi=contract_abi)

In [90]:
roles = ['municipality'] + [f'citizen_{i}' for i in range(1,6)] + [f'truck_{i}' for i in range(1,3)] + [f'station_{i}' for i in range(1,3)]    
address_dict = {roles[i]: web3.eth.accounts[i] for i in range(len(roles))}

In [91]:
address_dict

{'municipality': '0xCe38D1B5E5255023f8772363Bee10be9AAD5a663',
 'citizen_1': '0x0D15a21df8F04a1eadfA41eC5d07deBe8afCA264',
 'citizen_2': '0x7C4Ad0C5E20825D78305f80C1E0F361186127A70',
 'citizen_3': '0xd5e8dFB1DBF4e35ca7F0750971d4DdF4672735a7',
 'citizen_4': '0x85C95c7674b2c2569597396e6cb805B5a078c3A8',
 'citizen_5': '0x0f2d24B69f8A96a8f436D9dCE3B9616A0BE404Fe',
 'truck_1': '0xF51f7899c9fc3AC8E5b71feD71A9c38a79B1e8fc',
 'truck_2': '0x854Dd3026ABB33e2bDCF781D4733D0e4EBfA1991',
 'station_1': '0x24F2E663AC726438409B930569D9de6346a8CBeE',
 'station_2': '0x201F7945298E1E8EE3F5b2C30fee2a29F9Fa5457'}

In [92]:
# All functions in the contract
rubbish_contract.all_functions()

[<Function _addGarbageCollectorRole(address)>,
 <Function _addMunicipalityManager(address)>,
 <Function _removeCitizenRole(address)>,
 <Function _removeGarbageCollectorRole(address)>,
 <Function _removeMunicipalityManagers(address)>,
 <Function addCitizen()>,
 <Function didIPayDeposit()>,
 <Function generateTrashBag()>,
 <Function getAmountNonRecyclableWaste()>,
 <Function getAmountRecyclableWaste()>,
 <Function getTaxesDue()>,
 <Function getTotalTaxesPaid()>,
 <Function owner()>,
 <Function payDeposit()>,
 <Function pickFromBin(address,uint256,uint8)>,
 <Function renounceOwnership()>,
 <Function transferOwnership(address)>]

Add citizens to the citizen list using function `addCitizen()`

In [93]:
for citizen in [f'citizen_{i}' for i in range(1,6)]:
    print(citizen)
    tx_hash = rubbish_contract.functions.addCitizen().transact({'from': address_dict[citizen]})

citizen_1
citizen_2
citizen_3
citizen_4
citizen_5


In [94]:
def get_info_from_citizen(contract, citizen):
    """
    Given a contract and a citizen, retrieve information regarding taxes paid/due and rubbish produced
    
    citizen must be a str
    
    Taxes paid -> only municiplaity manager can access this!
    """
    
    from_dict = {'from': address_dict[citizen]}
    paid_deposit = contract.functions.didIPayDeposit().call(from_dict)
    nonrec_waste_amount = contract.functions.getAmountNonRecyclableWaste().call(from_dict)
    rec_waste_amount = contract.functions.getAmountRecyclableWaste().call(from_dict)
    taxes_due = contract.functions.getTaxesDue().call(from_dict)
    #taxes_paid = contract.functions.getTotalTaxesPaid().call(from_dict)
    return paid_deposit, nonrec_waste_amount, rec_waste_amount, taxes_due, #taxes_paid

In [95]:
c1 = get_info_from_citizen(rubbish_contract, 'citizen_1')
c1

(False, 0, 0, 0)

In [106]:
# Minimum deposit
deposit = 
tx_hash = rubbish_contract.functions.payDeposit(deposit).transact({'from': address_dict['citizen_1']})

ValidationError: 
Could not identify the intended function with name `payDeposit`, positional argument(s) of type `(<class 'str'>,)` and keyword argument(s) of type `{}`.
Found 1 function(s) with the name `payDeposit`: ['payDeposit()']
Function invocation failed due to improper number of arguments.

In [12]:
import time

def handle_event(event):
    print(event)

def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(poll_interval)

def main():
    block_filter = web3.eth.filter('latest')
    log_loop(block_filter, 2)

In [13]:
main()

b'\x94N\xd7\xf91\xaau\x89\xff\xe6\xe1m\xe4\xa1\xeb\xa1\xddU\xf2\xc7\x85\x07\xd9b\x076\xdf\xa7\xb9\xf6F('
b'\x14\xe3\x0f\xd5c\xc0)>\xb1\xa8%\r\xee\xd7\xdd\xd6\x82I\xfc\xc2k\xf1-\xa8\xb1\x9d;\x16\xfb\xb5\xe3-'


KeyboardInterrupt: 